## Load BOM

In [1]:
import os
import pandas as pd
import numpy as np
import requests
import time as t
import json
from tqdm import tqdm
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# from dotenv import load_dotenv
# load_dotenv()

BOM_path = f"../DAC_GainOutputStage7.csv"

BOM_df = pd.read_csv(BOM_path, delimiter = ";")
BOM_df.drop(columns = 'Unnamed: 16', inplace = True)
display(BOM_df)
print("Done")

,Qty,Value,Device,Package,Parts,Description,ARROW_PART_NUMBER,ARROW_PRICE-STOCK,DESCRIPTION,HEIGHT,MANUFACTURER,MANUFACTURER_NAME,MANUFACTURER_PART_NUMBER,MOUSER_PART_NUMBER,MOUSER_PRICE-STOCK,PROD_ID
0,4,NaN,"BZX79-C16,113",DIOAD1165W56L425D185,"Z1, Z2, Z5, Z6","Nexperia Zener Diode, 16V +/-2 %, +/-5 % 500 m...",NaN,NaN,"Nexperia Zener Diode, 16V +/-2 %, +/-5 % 500 m...",mm,NaN,Nexperia,"BZX79-C16,113",NaN,NaN,NaN
1,2,NaN,CONN_06SILK_FEMALE_PTH,1X06,"J2, J13",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONN-08437
2,2,"0.1uF, 35V",MKS4B031002A00JC00,MKS4B031002A00JC00,"C2O, C2O1",Film Capacitors MKS 4 0.1 uF 50 VDC 2.5x7x10 P...,NaN,NaN,Film Capacitors MKS 4 0.1 uF 50 VDC 2.5x7x10 P...,7mm,NaN,WIMA,MKS4B031002A00JC00,505-MKS4B0312AJC,https://www.mouser.co.uk/ProductDetail/WIMA/MK...,NaN
3,8,100uF,UFW1H101MPD1TD,CAPPRD350W60D825H1300,"C11O, C12O, C13O, C14O, C15O, C24O, C25O, C26O",Aluminum Electrolytic Capacitors - Radial Lead...,UFW1H101MPD1TD,https://www.arrow.com/en/products/ufw1h101mpd1...,Aluminum Electrolytic Capacitors - Radial Lead...,13mm,NaN,Nichicon,UFW1H101MPD1TD,647-UFW1H101MPD1TD,https://www.mouser.co.uk/ProductDetail/Nichico...,NaN
4,10,10k,LR1F10K,0207/10,"R11O, R19O, R20O, R20O1, R20O2, R21O, R22O, R2...","Metal film resistor, 10K 0.6W TE Connectivity ...",NaN,NaN,"Metal film resistor, 10K 0.6W TE Connectivity ...",mm,NaN,TE Connectivity,LR1F10K,279-LR1F10K,https://www.mouser.co.uk/ProductDetail/TE-Conn...,NaN
5,4,15k,CPF215K000FKE14,0411/15,"R5, R11, R19, R20",VISHAY - CPF215K000FKE14 - Through Hole Resist...,NaN,NaN,VISHAY - CPF215K000FKE14 - Through Hole Resist...,mm,NaN,Vishay,CPF215K000FKE14,71-CPF2-F-15K-E3,https://www.mouser.com/ProductDetail/71-CPF2-F...,NaN
6,8,182,MF0207FTE52-182R,0207/10,"R12O, R14O, R16O, R18O, R23O, R24O, R25O, R26O",Metal Film Resistors - Through Hole 0.6W 1% 18...,NaN,NaN,Metal Film Resistors - Through Hole 0.6W 1% 18...,mm,Yageo,NaN,MF0207FTE52-182R,603-MF0207FTE52-182R,https://www.mouser.com/ProductDetail/YAGEO/MF0...,NaN
7,1,1935190,1935190,1935190,J1,"PCB terminal block, nominal current: 17.5 A, r...",NaN,NaN,"PCB terminal block, nominal current: 17.5 A, r...",11.55mm,NaN,Phoenix Contact,1935190,651-1935190,https://www.mouser.co.uk/ProductDetail/Phoenix...,NaN
8,2,1M*,LR1F1M0,0207/10,"R3O, R3O1",TE Connectivity Axial Metal Film Resistor 1m +...,NaN,NaN,TE Connectivity Axial Metal Film Resistor 1m +...,mm,NaN,TE Connectivity,LR1F1M0,279-LR1F1M0,https://www.mouser.co.uk/ProductDetail/TE-Conn...,NaN
9,6,1N4148-T50R,1N4148-T50R,DIOAD1068W53L380D172,"D1, D2, D3, D4, D5, D6","Diodes - General Purpose, Power, Switching Hi ...",NaN,NaN,"Diodes - General Purpose, Power, Switching Hi ...",mm,NaN,onsemi,1N4148-T50R,512-1N4148T50R,https://www.mouser.co.uk/ProductDetail/onsemi-...,NaN


Done


## Define search_part function

In [2]:
def search_part(part_name):
    MOUSER_API_KEY = "c7b47461-c777-4dbc-8413-9bb69528b78f"
    
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }
    
    params = {
        'apiKey': MOUSER_API_KEY,
    }
    
    json_data = {
        'SearchByPartRequest': {
            'mouserPartNumber': part_name,
            'partSearchOptions': 'string',
            'mouserPaysCustomsAndDuties': False,
        },
    }

    try_again_flag = True
    max_attempts = 10
    attempts = 0
    
    while try_again_flag and attempts < max_attempts:
        response = requests.post('https://api.mouser.com/api/v1/search/partnumber', params=params, headers=headers, json=json_data)
    
        t.sleep(6)
    
        match response.status_code:
            case 200:
                data = response.json()
                try_again_flag = False
            case 404:
                print("Resource not found (404 Not Found).")
                attempts += 1
            case 403:
                print("Permission denied (403 Forbidden).")
                attempts += 1
            case response.status_code if 300 <= response.status_code < 400:
                print(f"Redirection occurred (Status: {response.status_code}).")
                attempts += 1
            case response.status_code if 500 <= response.status_code < 600:
                print(f"Server error occurred (Status: {response.status_code}).")
                attempts += 1
            case _:
                print(f"Unexpected status code: {response.status_code} - {response.text}")
                attempts += 1

    if attempts == max_attempts:
        print(f"Max attempts reached - {attempts} / {max_attempts}.")
        return
    
    df = pd.DataFrame()
    
    # Sort and append new data
    for part in data["SearchResults"]["Parts"]:
        df = pd.concat([df, pd.json_normalize(part)], ignore_index=True)

    columns_to_drop = ["ImagePath",
                     "AlternatePackagings",
                     "ProductDetailUrl",
                     "Reeling",
                     "MultiSimBlue",
                     "InfoMessages",
                     "SurchargeMessages",
                     "ProductCompliance",
                     "REACH-SVHC",
                     "RestrictionMessage",
                     "ROHSStatus",
                     "ProductAttributes",
                     "FactoryStock",
                     "AvailabilityOnOrder",
                     "UnitWeightKg.UnitWeight",
                    ]

    for column_to_drop in columns_to_drop:
        if column_to_drop in df.columns:
            df.drop([column_to_drop], axis = 1, inplace = True)
        else:
            pass 

    if "PriceBreaks" in df.columns:
        df_exploded = df.explode("PriceBreaks").reset_index(drop=True)
        pricing_df = pd.json_normalize(df_exploded["PriceBreaks"])
        df_combined = pd.concat([df_exploded.drop(columns=["PriceBreaks"]), pricing_df], axis=1)
        df = df_combined.set_index(["MouserPartNumber", "Quantity"]).sort_index()
    else:
        return df
    
    for column in df.columns:
        try:
            df[column] = pd.to_numeric(df[column])
        except Exception as e:
            pass
    
    df = df[df["AvailabilityInStock"] > 0]
    df = df[df["Min"] == 1]
    df = df[df["Mult"] == 1]    
    df = df.sort_values(by=['Quantity', 'Price'])
    return df.loc[[df.index.get_level_values(0)[0]]]

print("Done")

Done


## Search Parts

In [3]:
results = pd.DataFrame()
pared_BOM = pd.DataFrame()
missing_parts = pd.DataFrame()

with tqdm(total = len(BOM_df["MOUSER_PART_NUMBER"].values), unit = "part") as pbar:
    for idx, row in BOM_df.iterrows():
        if row["MOUSER_PART_NUMBER"] is not np.nan:
            part_result = search_part(row["MOUSER_PART_NUMBER"])
        else:
            part_result = search_part(row["Device"])
            
        if part_result.empty:
            missing_parts = pd.concat([missing_parts, row], axis = 1)
        else:
            results = pd.concat([results, pd.concat([part_result.reset_index(), pd.concat([pd.DataFrame(row).T] * len(part_result), ignore_index=True)], axis = 1)], axis = 0)
        pbar.update(1)

results = results.set_index(["MouserPartNumber", "Quantity"]).sort_index()
display(results)
print("Cant find part(s):")
display(missing_parts.T)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [02:11<00:00,  6.60s/part]


Availability DataSheetUrl  \
MouserPartNumber Quantity                                
279-LR1F10K      1         47362 In Stock          NaN   
                 10        47362 In Stock          NaN   
                 25        47362 In Stock          NaN   
                 100       47362 In Stock          NaN   
                 250       47362 In Stock          NaN   
...                                   ...          ...   
771-BZX79-C16113 100       24368 In Stock          NaN   
                 500       24368 In Stock          NaN   
                 1000      24368 In Stock          NaN   
                 5000      24368 In Stock          NaN   
                 10000     24368 In Stock          NaN   

                                                                 Description  \
MouserPartNumber Quantity                                                      
279-LR1F10K      1         Metal Film Resistors - Through Hole 10KOhm 3/5...   
                 10        Metal Film Resistors - Through Hole 10KOhm 3/5...   
                 25        Metal Film Resistors - Through Hole 10KOhm 3/5...   
                 100       Metal Film Resistors - Through Hole 10KOhm 3/5...   
                 250       Metal Film Resistors - Through Hole 10KOhm 3/5...   
...                                                                      ...   
771-BZX79-C16113 100              Zener Diodes ZENER   SOD-27   .5W      16V   
                 500              Zener Diodes ZENER   SOD-27   .5W      16V   
                 1000             Zener Diodes ZENER   SOD-27   .5W      16V   
                 5000             Zener Diodes ZENER   SOD-27   .5W      16V   
                 10000            Zener Diodes ZENER   SOD-27   .5W      16V   

                                                      Category LeadTime  \
MouserPartNumber Quantity                                                 
279-LR1F10K      1         Metal Film Resistors - Through Hole  66 Days   
                 10        Metal Film Resistors - Through Hole  66 Days   
                 25        Metal Film Resistors - Through Hole  66 Days   
                 100       Metal Film Resistors - Through Hole  66 Days   
                 250       Metal Film Resistors - Through Hole  66 Days   
...                                                        ...      ...   
771-BZX79-C16113 100                              Zener Diodes  56 Days   
                 500                              Zener Diodes  56 Days   
                 1000                             Zener Diodes  56 Days   
                 5000                             Zener Diodes  56 Days   
                 10000                            Zener Diodes  56 Days   

                          LifecycleStatus                  Manufacturer  \
MouserPartNumber Quantity                                                 
279-LR1F10K      1                    NaN  TE Connectivity / Holsworthy   
                 10                   NaN  TE Connectivity / Holsworthy   
                 25                   NaN  TE Connectivity / Holsworthy   
                 100                  NaN  TE Connectivity / Holsworthy   
                 250                  NaN  TE Connectivity / Holsworthy   
...                                   ...                           ...   
771-BZX79-C16113 100                  NaN                      Nexperia   
                 500                  NaN                      Nexperia   
                 1000                 NaN                      Nexperia   
                 5000                 NaN                      Nexperia   
                 10000                NaN                      Nexperia   

                          ManufacturerPartNumber  Min  Mult  ...  \
MouserPartNumber Quantity                                    ...   
279-LR1F10K      1                       LR1F10K    1     1  ...   
                 10                      LR1F10K    1     1  ...   
         

Cant find part(s):


,Qty,Value,Device,Package,Parts,Description,ARROW_PART_NUMBER,ARROW_PRICE-STOCK,DESCRIPTION,HEIGHT,MANUFACTURER,MANUFACTURER_NAME,MANUFACTURER_PART_NUMBER,MOUSER_PART_NUMBER,MOUSER_PRICE-STOCK,PROD_ID
1,2,NaN,CONN_06SILK_FEMALE_PTH,1X06,"J2, J13",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONN-08437
16,2,LM317MTG,LM317MTG,TO254P482X997X2018-3P,"U1, U2",Internal Short Circuit Current Limiting; Inter...,NaN,NaN,Internal Short Circuit Current Limiting,Internal Thermal Overload Protection,Output Current in Excess of 500 mA,Output Transistor Safe-Area Compensation,Output Adjustable between 1.2 V and 37 V,Floating Operation for High Voltage Applications,Eliminates Stocking Many Fixed Voltages,Pb-Free Packages are Available
19,4,VT9-PT,VT9-PT,VT9-PT,"SX1, SX2, SX3, SX4",VACUUM TUBE SOCKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Create Output Dataframe

In [21]:
quantities = np.unique([j for i, j in results.index.values])
quantities = quantities[quantities <= 100]

price_breaks = pd.DataFrame(np.unique([i for i, j in results.index.values]), columns = ["MouserPartNumber"])
price_breaks.set_index("MouserPartNumber", inplace = True)

for part_num in [i for i, j in results.index.values]:
    for quantity in quantities:
        result_df = results.loc[(results.index.get_level_values('MouserPartNumber') == part_num) & (results.index.get_level_values('Quantity') == quantity)]["Price"]
        if result_df.empty:
            price_breaks.loc[part_num, f"Unit Price Per {quantity}"] = np.nan
        else:
            price_breaks.loc[part_num, f"Unit Price Per {quantity}"] = result_df.values
            
price_breaks = price_breaks.ffill(axis=1)

# Remove dollar signs
for idx, row in price_breaks.iterrows(): # Bear in mind that "idx" is the part name!!!
    for col_name in price_breaks.columns:
        row[col_name] = str(row[col_name]).replace('$', '')

for column in price_breaks.columns:
    try:
        price_breaks[column] = pd.to_numeric(price_breaks[column])
    except Exception as e:
        pass

# Add in columns from BOM
in_cols = ["Qty", "Value", "Device"]
out_cols = ["Per 1 Qty", "Value", "Device"]

for idx1, (in_col, out_col) in enumerate(zip(in_cols, out_cols)):
    for idx2, part_num in enumerate(price_breaks.index.values):
        qty_val = np.unique(results.loc[results.index.get_level_values('MouserPartNumber') == part_num][in_col].values)[0]
    
        if idx2 == 0:
            price_breaks.insert(loc = idx1, column = out_col, value = qty_val)
        else:
            price_breaks.at[part_num, out_col] = qty_val

# Qty
multiplier = 2
price_breaks.insert(loc = 1, column = "Qty", value = [i * multiplier for i in price_breaks["Per 1 Qty"].values])

# Extras?
extras = [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
price_breaks.at[:, "Extras?"] = extras

# Sub-count
for idx, row in price_breaks.iterrows():
    if row["Extras?"] and (row["Unit Price Per 1"] < 1):
        price_breaks.at[idx, "Sub-count"] = np.round(row["Qty"] + 1 + row["Qty"] * 0.03)
    else:
        price_breaks.at[idx, "Sub-count"] = row["Qty"]

# Sub-totals per number
in_cols = ["Unit Price Per 1",  "Unit Price Per 10", "Unit Price Per 20", "Unit Price Per 25", "Unit Price Per 50", "Unit Price Per 100"]
out_cols = ["Sub-total <5", "Sub-total Per 10", "Sub-total Per 20", "Sub-total Per 25", "Sub-total Per 50", "Sub-total Per 100"]

for _, (in_col, out_col) in enumerate(zip(in_cols, out_cols)):
    for idx, row in price_breaks.iterrows():
        multiplier = int(in_col.replace("Unit Price Per ", ""))
        price_breaks.at[idx, out_col] = multiplier * row[in_col]

# Final Count
#=IF(L3<XLOOKUP(MIN(M3:R3),M3:R3,M$1:R$1,,,-1),XLOOKUP(MIN(M3:R3),M3:R3,M$1:R$1,,,-1),L3)

display(price_breaks)

,Per 1 Qty,Qty,Value,Device,Unit Price Per 1,Unit Price Per 10,Unit Price Per 20,Unit Price Per 25,Unit Price Per 50,Unit Price Per 100,Extras?,Sub-count,Sub-total <5,Sub-total Per 10,Sub-total Per 20,Sub-total Per 25,Sub-total Per 50,Sub-total Per 100
MouserPartNumber,,,,,,,,,,,,,,,,,,
279-LR1F10K,10,20,10k,LR1F10K,0.10,0.027,0.027,0.024,0.024,0.023,True,22.0,0.10,0.27,0.54,0.600,1.20,2.3
279-LR1F1K0,8,16,1k,LR1F1K0,0.10,0.030,0.030,0.027,0.027,0.025,True,17.0,0.10,0.30,0.60,0.675,1.35,2.5
279-LR1F1M0,2,4,1M*,LR1F1M0,0.10,0.027,0.027,0.024,0.024,0.023,True,5.0,0.10,0.27,0.54,0.600,1.20,2.3
279-YR1B76R8CC,2,4,76.8,YR1B76R8CC,1.04,0.636,0.636,0.516,0.516,0.370,True,4.0,1.04,6.36,12.72,12.900,25.80,37.0
490-SJ-63033A,1,2,SJ-63033A,SJ-63033A,1.99,1.700,1.590,1.590,1.520,1.430,True,2.0,1.99,17.00,31.80,39.750,76.00,143.0
494-1N750(DO-35),2,4,1N750,1N750,2.02,2.020,2.020,2.020,2.020,1.870,True,4.0,2.02,20.20,40.40,50.500,101.00,187.0
505-DCP4G0536ID2KSSD,2,4,30uF/400V,DCP4G053006ID2KSSD,9.32,6.970,6.970,6.970,6.970,5.520,True,4.0,9.32,69.70,139.40,174.250,348.50,552.0
505-MKP4G034705DKSSD,4,8,MKP4G034705D00KSSD,MKP4G034705D00KSSD,1.37,0.939,0.939,0.939,0.939,0.795,True,8.0,1.37,9.39,18.78,23.475,46.95,79.5
505-MKS4B0312AJC,2,4,"0.1uF, 35V",MKS4B031002A00JC00,0.69,0.462,0.462,0.462,0.462,0.315,True,5.0,0.69,4.62,9.24,11.550,23.10,31.5
